DROP TABLE IF EXISTS DWA_d_datum;
CREATE TABLE DWA_d_datum(
    datum Date
    ,jaar Integer
    ,maand Integer
    ,seizoen Varchar(10)
    ,winterstop Varchar (5)
    );
    
DROP TABLE IF EXISTS DWA_d_teams;
CREATE TABLE DWA_d_teams(
    team Varchar (100)
    ,team_fd Varchar (100)
    ,team_oddapi Varchar (100)
    ,blessure Integer
    ,schorsingen Integer
    );

DROP TABLE IF EXISTS DWA_d_wedkantoren;
CREATE TABLE DWA_d_wedkantoren(
    wedkantoor Varchar (100)
    ,wk_fd Varchar (100)
    ,wk_oddapi Varchar (100)
    ,beschikbaar_NL Boolean
    ,actief Boolean
    );

DROP TABLE IF EXISTS DWA_d_wedstrijden;
CREATE TABLE DWA_d_wedstrijden(
    s_wedstrijd Integer
    ,competitie Varchar (10)
    ,hometeam Varchar(100)
    ,awayteam Varchar(100)
    );

DROP TABLE IF EXISTS DWA_f_resultaten;
CREATE TABLE DWA_f_resultaten(
    s_wedstrijd Integer
    ,date Date
    ,fthg Integer
    ,ftag Integer
    ,ftr Varchar(1)
    ,hthg Integer
    ,htag Integer
    ,htr Varchar(1)
    ,hsh Real
    ,ash Real
    ,hst Real
    ,ast Real
    ,hf Real
    ,af Real
    ,hc Real
    ,ac Real
    ,hy Real
    ,ay Real
    ,hr Real
    ,ar Real
    );

DROP TABLE IF EXISTS DWA_f_odds;
CREATE TABLE DWA_f_odds(
    s_wedstrijd Integer
    ,wedkantoor Varchar (100)
    ,odd_datum Date
    ,odd_type Varchar(10)
    ,odd_prijs Decimal
    );

DROP VIEW IF EXISTS DWA_f_vorm_VIEW;
CREATE VIEW DWA_f_vorm_VIEW AS
WITH wedstrijden AS (
	SELECT dd.seizoen, hometeam, awayteam, fr.s_wedstrijd, date, fthg, ftag, hsh, ash
	from DWA_f_resultaten fr
	LEFT OUTER JOIN DWA_d_wedstrijden dw 
		ON fr.s_wedstrijd = dw.s_wedstrijd
	left outer join DWA_d_datum dd 
		on fr.date = dd.datum 
    )
SELECT 
row_number() over (partition BY team, seizoen order by date desc) nr
,*
,LAG(GRW, 0) OVER (partition BY team, seizoen order by date) +
 LAG(GRW, 1) OVER (partition BY team, seizoen order by date) +
 LAG(GRW, 2) OVER (partition BY team, seizoen order by date) +
 LAG(GRW, 3) OVER (partition BY team, seizoen order by date) AS GR -- Goalratio
,LAG(SRW, 0) OVER (partition BY team, seizoen order by date) +
 LAG(SRW, 1) OVER (partition BY team, seizoen order by date) +
 LAG(SRW, 2) OVER (partition BY team, seizoen order by date) +
 LAG(SRW, 3) OVER (partition BY team, seizoen order by date) AS SR -- Shotratio
 FROM (
		SELECT 
		hometeam AS team
		,fthg - ftag AS GRW -- GoalRatio Wedstrijd
		,hsh - ash as SRW	-- ShotRatio Wedstrijd
		,* 
		FROM wedstrijden
		UNION
		SELECT awayteam AS team
		,-1*(fthg - ftag) AS GRW -- GoalRatio Wedstrijd
		,-1*(hsh - ash) as SRW	-- ShotRatio Wedstrijd
		,* 
		FROM wedstrijden
		ORDER BY team, date
		) GR
;

DROP TABLE IF EXISTS DWA_f_bethistorie;
CREATE TABLE DWA_f_bethistorie(
speelronde integer
,datum date
,s_wedstrijd varchar (20)
,best_odd decimal
,min_odd decimal
,rea_inzet decimal
,rea_odd decimal
,wedkantoor varchar (50)
);

DROP VIEW IF EXISTS DWA_betresultaten_VIEW;
create view DWA_betresultaten_VIEW as 
select 
bh.speelronde,
dw.competitie,
dd.seizoen,
bh.datum,
dw.hometeam,
dw.awayteam,
fr.fthg,
fr.ftag,
fr.ftr,
bh.best_odd,
bh.min_odd,
bh.rea_inzet,
bh.rea_odd,
case when fr.ftr = 'H' then 1 else 0 end wins
from DWA_f_bethistorie bh
left outer join DWA_d_wedstrijden dw 
	on bh.s_wedstrijd = dw.s_wedstrijd
left outer join DWA_d_datum dd 
	on bh.datum = dd.datum
left outer join DWA_f_resultaten fr 
	on bh.datum = fr.date and bh.s_wedstrijd = fr.s_wedstrijd
;
